<a href="https://colab.research.google.com/github/victorviro/Deep_learning_python/blob/master/Reusing_pretrained_layers_DNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In previous notebooks, we introduced the artificial neural networks and trained our first deep neural networks. But they were shallow nets, with just a few hidden layers. What if we need to tackle a complex problem, such as detecting hundreds of types of objects in high-resolution images? We may need to train a much deeper DNN, perhaps with 10 layers or many more, each containing hundreds of neurons, linked by hundreds of thousands of connections. Training a deep DNN isn’t a walk in the park. Here are some of the problems we could run into:

- We may be faced with the tricky *vanishing gradients* problem or the related *exploding gradients* problem. This is when the gradients grow smaller and smaller, or larger and larger when flowing backward through the DNN during training. Both of these problems make lower layers very hard to train (see notebook [The vanishing/exploding gradients problem](https://nbviewer.jupyter.org/github/victorviro/Deep_learning_python/blob/master/Vanishing_Exploding_gradients_problem_DNNs.ipynb)).

- We might not have enough training data for such a large network, or it might be too costly to label.

- Training may be extremely slow.

- A model with millions of parameters would severely risk overfitting the training set, especially if there are not enough training instances or if they are too noisy.

In this notebook, we will look at transfer learning and unsupervised pretraining, which can help us tackle complex tasks even when we have little labeled data.

## Reusing Pretrained Layers

It is generally not a good idea to train a very large DNN from scratch: instead, we should always try to find an existing neural network that accomplishes a similar task to the one we are trying to tackle, then reuse the lower layers of this network. This technique is called *transfer learning*. It will not only speed up training considerably, but also require significantly less training data.

Suppose we have access to a DNN that was trained to classify pictures into 100 different categories, including animals, plants, vehicles, and everyday objects. We now want to train a DNN to classify specific types of vehicles. These tasks are slightly similar, even partly overlapping, so we should try to reuse parts of the first network (see Figure 11-4).

![texto alternativo](https://i.ibb.co/nCjDHHb/reuse-pretrained-layers.png)

**Note**: If the input pictures of our new task don’t have the same size as the ones used in the original task, we will usually have to add a preprocessing step to resize them to the size expected by the original model. More generally, transfer learning will work best when the inputs have similar low-level features.

The output layer of the original model should usually be replaced because it is most likely not useful at all for the new task, and it may not even have the right number of outputs for the new task.

Similarly, the upper hidden layers of the original model are less likely to be as useful as the lower layers, since the high-level features that are most useful for the new task may differ significantly from the ones that were most useful for the original task. We want to find the right number of layers to reuse.

**Note**: The more similar the tasks are, the more layers we can to reuse (starting with the lower layers). For very similar tasks, we can try keeping all the hidden layers and just replacing the output layer.

We try freezing all the reused layers first (i.e., make their weights non-trainable so that Gradient Descent won’t modify them), then we train our model and see how it performs. Then we try unfreezing one or two of the top hidden layers to let backpropagation tweak them and see if performance improves. The more training data we have, the more layers we can unfreeze. It is also useful to reduce the learning rate when we unfreeze reused layers: this will avoid wrecking their fine-tuned weights.

If we still cannot get good performance, and we have little training data, we can try dropping the top hidden layer(s) and freezing all the remaining hidden layers again. We can iterate until we find the right number of layers to reuse. If we have plenty of training data, we may try replacing the top hidden layers instead of dropping them, and even adding more hidden layers.

In a coming notebook we will revisit transfer learning to train an image classifier using convolutional neural networks, but now let's see some strategies we can follow if we can't get a model trained in a similar task.

### Unsupervised Pretraining



Suppose we want to tackle a complex task for which we don’t have much labeled training data, but unfortunately, we cannot find a model trained on a similar task. First, we should try to gather more labeled training data, but if we can’t, we may still be able to perform *unsupervised pretraining* (see Figure 11-5). Indeed, it is often cheap to gather unlabeled training examples, but expensive to label them. If we can gather plenty of unlabeled training data, we can try to use it to train an unsupervised model, such as an autoencoder or a generative adversarial network. Then we can reuse the lower layers of the autoencoder or the lower layers of the GAN’s discriminator, add the output layer for our task on top, and fine-tune the final network using supervised learning (i.e., with the labeled training examples).

![texto alternativo](https://i.ibb.co/NTGdKc1/unsupervides-pretraining.png)

It is this technique that Geoffrey Hinton and his team used in 2006 and which led to the revival of neural networks and the success of Deep Learning. Until 2010, unsupervised pretraining—typically with [restricted Boltzmann machines](https://en.wikipedia.org/wiki/Restricted_Boltzmann_machine) (RBMs) was the norm for deep nets, and only after the vanishing gradients problem was alleviated did it become much more common to train DNNs purely using supervised learning. Unsupervised pretraining (today typically using autoencoders or GANs rather than RBMs) is still a good option when we have a complex task to solve, no similar model we can reuse, and little labeled training data but plenty of unlabeled training data.

Note that in the early days of Deep Learning it was difficult to train deep models, so people would use a technique called greedy layer-wise pretraining (depicted in Figure 11-5). They would first train an unsupervised model with a single layer, typically an RBM, then they would freeze that layer and add another one on top of it, then train the model again (effectively just training the new layer), then freeze the new layer and add another layer on top of it, train the model again, and so on. Nowadays, things are much simpler: people generally train the full unsupervised model in one shot (i.e., in Figure 11-5, just start directly at step three) and use autoencoders or GANs rather than RBMs.

### Pretraining on an Auxiliary Task

If we do not have much labeled training data, one last option is to train a first neural network on an auxiliary task for which we can easily obtain or generate labeled training data, then reuse the lower layers of that network for our actual task. The first neural network’s lower layers will learn feature detectors that will likely be reusable by the second neural network.

For example, if we want to build a system to recognize faces, we may only have a few pictures of each individual—clearly not enough to train a good classifier. Gathering hundreds of pictures of each person would not be practical. We could, however, gather a lot of pictures of random people on the web and train a first neural network to detect whether or not two different pictures feature the same person. Such a network would learn good feature detectors for faces, so reusing its lower layers would allow you to train a good face classifier that uses little training data.

For *natural language processing* (NLP) applications, we can download a corpus of millions of text documents and automatically generate labeled data from it. For example, we could randomly mask out some words and train a model to predict what the missing words are (e.g., it should predict that the missing word in the sentence "What ___ you saying?" is probably "are" or "were"). If we can train a model to reach good performance on this task, then it will already know quite a lot about language, and we can certainly reuse it for our actual task and fine-tune it on our labeled data (see the notebook [Bidirectional Encoder Representations for Transformers (BERT)](https://nbviewer.jupyter.org/github/victorviro/Deep_learning_python/blob/master/BERT.ipynb)). 


### Self-supervised learning

*Self-supervised learning* is when we automatically generate the labels from the data itself, then we train a model on the resulting "labeled" dataset using supervised learning techniques. Since this approach requires no human labeling whatsoever, it is best classified as a form of unsupervised learning.

# References

- [Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/)

- https://github.com/ageron/handson-ml2

- [A Survey on Deep Transfer Learning](https://arxiv.org/abs/1808.01974)

- [A Comprehensive Survey on Transfer Learning](https://arxiv.org/abs/1911.02685)